## WeGo Data Introduction

In [339]:
import pandas as pd

In [341]:
import numpy as np

In [152]:
wego = pd.read_csv("../data/Route 50 Timepoint and Headway Data, 1-1-2023 through 5-12-2025.csv")

All the data that you have been provided is from Route 50, Charlotte Pike.

In [154]:
wego['ROUTE_ABBR'].value_counts().sort_index()

ROUTE_ABBR
50    618998
Name: count, dtype: int64

The trip can be identified by the DATE/CALENDAR_ID plus the TRIP_ID.  
**Warning:** The TRIP_ID refers to the route and time but will be used across multiple days.

The data contains multiple **time points** for each trip. There are more stops along the route than time points, but the time points are the points with specific scheduled times the bus operators must adhere to.

The first stop of a trip has a TRIP_EDGE of 1, the last has a TRIP_EDGE of 2, and the intermediate stops are TRIP_EDGE 0. 

Here is the first trip in the dataset. It occurred on January 1, 2023 and was scheduled to start at 5:34 and end at 6:05.

In [156]:
(
    wego
    .loc[wego['CALENDAR_ID'] == 120230101]
    .loc[wego['TRIP_ID'] == 332422]
    [['DATE', 'CALENDAR_ID', 'TRIP_ID', 'ROUTE_ABBR', 'TIME_POINT_ABBR', 'TRIP_EDGE', 'SCHEDULED_TIME']]
)

,DATE,CALENDAR_ID,TRIP_ID,ROUTE_ABBR,TIME_POINT_ABBR,TRIP_EDGE,SCHEDULED_TIME
0,2023-01-01,120230101,332422,50,WALM,1,05:34:00
1,2023-01-01,120230101,332422,50,HLWD,0,05:40:00
2,2023-01-01,120230101,332422,50,WHBG,0,05:47:00
3,2023-01-01,120230101,332422,50,CH46,0,05:50:00
4,2023-01-01,120230101,332422,50,28&CHARL,0,05:54:00
5,2023-01-01,120230101,332422,50,MCC5_1,2,06:05:00


Note that the same TRIP_ID appears on the following day with a different CALENDAR_ID.

In [158]:
(
    wego
    .loc[wego['CALENDAR_ID'] == 120230102]
    .loc[wego['TRIP_ID'] == 332422]
    [['DATE', 'CALENDAR_ID', 'TRIP_ID', 'ROUTE_ABBR', 'TIME_POINT_ABBR', 'TRIP_EDGE', 'SCHEDULED_TIME']]
)

,DATE,CALENDAR_ID,TRIP_ID,ROUTE_ABBR,TIME_POINT_ABBR,TRIP_EDGE,SCHEDULED_TIME
516,2023-01-02,120230102,332422,50,WALM,1,05:34:00
517,2023-01-02,120230102,332422,50,HLWD,0,05:40:00
518,2023-01-02,120230102,332422,50,WHBG,0,05:47:00
519,2023-01-02,120230102,332422,50,CH46,0,05:50:00
520,2023-01-02,120230102,332422,50,28&CHARL,0,05:54:00
521,2023-01-02,120230102,332422,50,MCC5_1,2,06:05:00


**Adherence** refers to the difference between scheduled time and the actual time that the bus departs from a stop.

A negative value for ADHERENCE indicates that the bus is late, and a positive indicates that the bus is early.

Generally, an adherence value less than -6 is considered late, and greater than 1 is considered early, but there are some exceptions. For example, a positive adherence for the end of a trip (TRIP_EDGE 2) is not considered early, since it is not a problem if a bus ends its trip early as long as it didn't pass other timepoints early along the way. You can check whether a trip was considered on-time, early, or late using the ADJUSTED_EARLY_COUNT, ADJUSTED_LATE_COUNT, and ADJUSTED_ONTIME_COUNT columns.

Here is an example of a trip where all time points would be considered to be on time. Notice that at the end of the trip, the bus was almost 3 minutes early, but was still counted as on-time since this was a trip edge of 2. Also, the bus departed more than 5 minutes late but was still considered to be on-time.

In [160]:
(
    wego
    .loc[wego['CALENDAR_ID'] == 120230101]
    .loc[wego['TRIP_ID'] == 332423]
    [[
        'DATE', 'CALENDAR_ID', 'TRIP_ID', 'ROUTE_ABBR',
        'TIME_POINT_ABBR', 'TRIP_EDGE',
        'SCHEDULED_TIME', 'ACTUAL_DEPARTURE_TIME', 'ADHERENCE',
        'ADJUSTED_EARLY_COUNT', 'ADJUSTED_LATE_COUNT', 'ADJUSTED_ONTIME_COUNT'
    ]]
)

,DATE,CALENDAR_ID,TRIP_ID,ROUTE_ABBR,TIME_POINT_ABBR,TRIP_EDGE,SCHEDULED_TIME,ACTUAL_DEPARTURE_TIME,ADHERENCE,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT
6,2023-01-01,120230101,332423,50,MCC5_1,1,06:15:00,06:20:12,-5.200000,0,0,1
7,2023-01-01,120230101,332423,50,28&CHARL,0,06:25:00,06:26:55,-1.916666,0,0,1
8,2023-01-01,120230101,332423,50,CH46,0,06:29:00,06:31:29,-2.483333,0,0,1
9,2023-01-01,120230101,332423,50,WHBG,0,06:33:00,06:35:12,-2.200000,0,0,1
10,2023-01-01,120230101,332423,50,HLWD,0,06:40:00,06:40:41,-0.683333,0,0,1
11,2023-01-01,120230101,332423,50,WALM,2,06:47:00,06:44:01,2.983333,0,0,1


Here's an example of another trip later that same day that was considered late at the first time point, departing almost 8 minutes behind schedule.

In [162]:
(
    wego
    .loc[wego['CALENDAR_ID'] == 120230101]
    .loc[wego['TRIP_ID'] == 332493]
    [[
        'DATE', 'CALENDAR_ID', 'TRIP_ID', 'ROUTE_ABBR',
        'TIME_POINT_ABBR', 'TRIP_EDGE',
        'SCHEDULED_TIME', 'ACTUAL_DEPARTURE_TIME', 'ADHERENCE',
        'ADJUSTED_EARLY_COUNT', 'ADJUSTED_LATE_COUNT', 'ADJUSTED_ONTIME_COUNT'
    ]]
)

,DATE,CALENDAR_ID,TRIP_ID,ROUTE_ABBR,TIME_POINT_ABBR,TRIP_EDGE,SCHEDULED_TIME,ACTUAL_DEPARTURE_TIME,ADHERENCE,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT
210,2023-01-01,120230101,332493,50,MCC5_1,1,14:35:00,14:42:44,-7.733333,0,1,0
211,2023-01-01,120230101,332493,50,28&CHARL,0,14:45:00,14:49:29,-4.483333,0,0,1
212,2023-01-01,120230101,332493,50,CH46,0,14:49:00,14:54:25,-5.416666,0,0,1
213,2023-01-01,120230101,332493,50,WHBG,0,14:53:00,14:57:02,-4.033333,0,0,1
214,2023-01-01,120230101,332493,50,HLWD,0,15:01:00,15:03:04,-2.066666,0,0,1
215,2023-01-01,120230101,332493,50,WALM,2,15:10:00,15:06:38,3.366666,0,0,1


This trip departed nearly 5 minutes early from the first time point, so that time point would be considered early.

In [164]:
(
    wego
    .loc[wego['CALENDAR_ID'] == 120230101]
    .loc[wego['TRIP_ID'] == 332431]
    [[
        'DATE', 'CALENDAR_ID', 'TRIP_ID', 'ROUTE_ABBR',
        'TIME_POINT_ABBR', 'TRIP_EDGE',
        'SCHEDULED_TIME', 'ACTUAL_DEPARTURE_TIME', 'ADHERENCE',
        'ADJUSTED_EARLY_COUNT', 'ADJUSTED_LATE_COUNT', 'ADJUSTED_ONTIME_COUNT'
    ]]
)

,DATE,CALENDAR_ID,TRIP_ID,ROUTE_ABBR,TIME_POINT_ABBR,TRIP_EDGE,SCHEDULED_TIME,ACTUAL_DEPARTURE_TIME,ADHERENCE,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT
54,2023-01-01,120230101,332431,50,MCC5_1,1,12:15:00,12:10:04,4.933333,1,0,0
55,2023-01-01,120230101,332431,50,28&CHARL,0,12:25:00,12:27:55,-2.916666,0,0,1
56,2023-01-01,120230101,332431,50,CH46,0,12:29:00,12:30:48,-1.800000,0,0,1
57,2023-01-01,120230101,332431,50,WHBG,0,12:34:00,12:33:37,0.383333,0,0,1
58,2023-01-01,120230101,332431,50,HLWD,0,12:42:00,12:41:42,0.300000,0,0,1
59,2023-01-01,120230101,332431,50,WALM,2,12:50:00,12:48:24,1.600000,0,0,1


**Headway** is the amount of time between a bus and the prior bus at the same stop. In the dataset, the amount of headway scheduled is contained in the SCHEDULED_HDWY column and indicates the difference between the scheduled time for a particular stop and the scheduled time for the previous bus on that same stop.

This dataset contains a column HDWY_DEV, which shows the amount of deviation from the scheduled headway. **Bunching** occurs when there is shorter headway than scheduled, which would appear as a negative HDWY_DEV value. **Gapping** is when there is more headway than scheduled and appears as a positive value in the HDWY_DEV column. Note that you can calculate headway deviation percentage as HDWY_DEV/SCHEDULED_HDWY. 

The generally accepted range of headway deviation is 50% to 150% of the scheduled headway, so if scheduled headway is 10 minutes, a headway deviation of up to 5 minutes would be acceptable (but not ideal).

Here, you can see consecutive trips at the same time point. Notice that the scheduled headway is based on the scheduled time and that the actual headway is based on the actual departure times.

In [166]:
(
    wego
    .loc[wego['ROUTE_DIRECTION_NAME'] == 'TO DOWNTOWN']
    .loc[wego['TIME_POINT_ABBR'] == 'CH46']
    [['DATE', 'TRIP_ID', 'TIME_POINT_ABBR','ROUTE_DIRECTION_NAME', 'TRIP_EDGE', 
      'SCHEDULED_TIME', 'SCHEDULED_HDWY',
      'ACTUAL_DEPARTURE_TIME', 'ACTUAL_HDWY', 'HDWY_DEV'
     ]]
    .sort_values(['DATE', 'SCHEDULED_TIME'])
    .iloc[:5]
)

,DATE,TRIP_ID,TIME_POINT_ABBR,ROUTE_DIRECTION_NAME,TRIP_EDGE,SCHEDULED_TIME,SCHEDULED_HDWY,ACTUAL_DEPARTURE_TIME,ACTUAL_HDWY,HDWY_DEV
3,2023-01-01,332422,CH46,TO DOWNTOWN,0,05:50:00,NaN,05:50:34,NaN,NaN
147,2023-01-01,332482,CH46,TO DOWNTOWN,0,06:20:00,30.0,06:19:11,28.616666,-1.383334
243,2023-01-01,332536,CH46,TO DOWNTOWN,0,06:50:00,30.0,06:50:14,31.050000,1.050000
15,2023-01-01,332424,CH46,TO DOWNTOWN,0,07:20:00,30.0,07:24:18,34.066666,4.066666
159,2023-01-01,332484,CH46,TO DOWNTOWN,0,07:50:00,30.0,07:51:15,26.950000,-3.050000


In [168]:
wego.columns.tolist()

['CALENDAR_ID',
 'SERVICE_ABBR',
 'ADHERENCE_ID',
 'DATE',
 'ROUTE_ABBR',
 'BLOCK_ABBR',
 'OPERATOR',
 'TRIP_ID',
 'OVERLOAD_ID',
 'ROUTE_DIRECTION_NAME',
 'TIME_POINT_ABBR',
 'ROUTE_STOP_SEQUENCE',
 'TRIP_EDGE',
 'LATITUDE',
 'LONGITUDE',
 'SCHEDULED_TIME',
 'ACTUAL_ARRIVAL_TIME',
 'ACTUAL_DEPARTURE_TIME',
 'ADHERENCE',
 'SCHEDULED_HDWY',
 'ACTUAL_HDWY',
 'HDWY_DEV',
 'ADJUSTED_EARLY_COUNT',
 'ADJUSTED_LATE_COUNT',
 'ADJUSTED_ONTIME_COUNT',
 'STOP_CANCELLED',
 'PREV_SCHED_STOP_CANCELLED',
 'IS_RELIEF',
 'BLOCK_STOP_ORDER',
 'DWELL_IN_MINS']

In [170]:
wego.head()

,CALENDAR_ID,SERVICE_ABBR,ADHERENCE_ID,DATE,ROUTE_ABBR,BLOCK_ABBR,OPERATOR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,...,ACTUAL_HDWY,HDWY_DEV,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT,STOP_CANCELLED,PREV_SCHED_STOP_CANCELLED,IS_RELIEF,BLOCK_STOP_ORDER,DWELL_IN_MINS
0,120230101,3,93549161,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,NaN,NaN,0,0,1,0,0.0,0,2,8.133333
1,120230101,3,93549162,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,NaN,NaN,0,0,1,0,0.0,0,5,0.000000
2,120230101,3,93549163,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,NaN,NaN,0,0,1,0,0.0,0,11,0.000000
3,120230101,3,93549164,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,NaN,NaN,0,0,1,0,0.0,0,13,0.000000
4,120230101,3,93549165,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,NaN,NaN,0,0,1,0,0.0,0,18,2.150000


In [172]:
dates = [120250203, 120250210, 120250428, 120250505, 120250512]

In [214]:
view = (
    wego
    .loc[wego['ROUTE_ABBR'] == 50]
    .loc[wego['CALENDAR_ID'].isin([120250203, 120250210, 120250428, 120250505, 120250512])]
    [['CALENDAR_ID',
 'SERVICE_ABBR',
 'DATE',
 'ROUTE_ABBR',
 'TRIP_ID',
 'OVERLOAD_ID',
 'ROUTE_DIRECTION_NAME',
 'ROUTE_STOP_SEQUENCE',
 'TRIP_EDGE',
 'SCHEDULED_TIME',
 'ACTUAL_ARRIVAL_TIME',
 'ACTUAL_DEPARTURE_TIME',
 'ADHERENCE',
 'SCHEDULED_HDWY',
 'ACTUAL_HDWY',
 'HDWY_DEV',
 'ADJUSTED_EARLY_COUNT',
 'ADJUSTED_LATE_COUNT',
 'ADJUSTED_ONTIME_COUNT'
        ]]
)

In [176]:
view.sort_values('ACTUAL_ARRIVAL_TIME', ascending = False)

,CALENDAR_ID,SERVICE_ABBR,DATE,ROUTE_ABBR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,ROUTE_STOP_SEQUENCE,TRIP_EDGE,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ACTUAL_DEPARTURE_TIME,ADHERENCE,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT
618862,120250512,1,2025-05-12,50,429643,0,FROM DOWNTOWN,13,1,1900-01-01 00:15:00,23:59:22,1900-01-01 00:16:28,-1.466666,30.0,28.200000,-1.800000,0,0,1
618861,120250512,1,2025-05-12,50,429642,0,TO DOWNTOWN,13,2,1900-01-01 00:05:00,23:59:22,23:59:22,5.633333,NaN,NaN,NaN,0,0,1
613594,120250505,1,2025-05-05,50,429575,0,FROM DOWNTOWN,15,0,23:57:00,23:59:14,23:59:14,-2.233333,30.0,30.566666,0.566666,0,0,1
613748,120250505,1,2025-05-05,50,429643,0,FROM DOWNTOWN,13,1,1900-01-01 00:15:00,23:59:06,1900-01-01 00:16:18,-1.300000,30.0,28.666666,-1.333334,0,0,1
613747,120250505,1,2025-05-05,50,429642,0,TO DOWNTOWN,13,2,1900-01-01 00:05:00,23:59:06,23:59:06,5.900000,NaN,NaN,NaN,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618563,120250512,1,2025-05-12,50,429523,0,FROM DOWNTOWN,14,0,1900-01-01 00:53:00,NaN,NaN,NaN,30.0,NaN,NaN,0,0,0
618564,120250512,1,2025-05-12,50,429523,0,FROM DOWNTOWN,15,0,1900-01-01 00:57:00,NaN,NaN,NaN,30.0,NaN,NaN,0,0,0
618565,120250512,1,2025-05-12,50,429523,0,FROM DOWNTOWN,16,0,1900-01-01 01:00:00,NaN,NaN,NaN,30.0,NaN,NaN,0,0,0
618566,120250512,1,2025-05-12,50,429523,0,FROM DOWNTOWN,17,0,1900-01-01 01:06:00,NaN,NaN,NaN,30.0,NaN,NaN,0,0,0


In [182]:
view['ACTUAL_ARRIVAL_TIME'] = view['ACTUAL_ARRIVAL_TIME'].str[-8:]

In [204]:
view['ACTUAL_ARRIVAL_TIME'] = pd.to_datetime(view['ACTUAL_ARRIVAL_TIME'], format='%X')

In [206]:
view.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4032 entries, 546973 to 618997
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   CALENDAR_ID            4032 non-null   int64         
 1   SERVICE_ABBR           4032 non-null   int64         
 2   DATE                   4032 non-null   object        
 3   ROUTE_ABBR             4032 non-null   int64         
 4   TRIP_ID                4032 non-null   int64         
 5   OVERLOAD_ID            4032 non-null   int64         
 6   ROUTE_DIRECTION_NAME   4032 non-null   object        
 7   ROUTE_STOP_SEQUENCE    4032 non-null   int64         
 8   TRIP_EDGE              4032 non-null   int64         
 9   SCHEDULED_TIME         4032 non-null   object        
 10  ACTUAL_ARRIVAL_TIME    3994 non-null   datetime64[ns]
 11  ACTUAL_DEPARTURE_TIME  3993 non-null   object        
 12  ADHERENCE              3993 non-null   float64       
 13  S

In [222]:
start_time = '12:00:00'
end_time = '23:59:59'

In [224]:
tsp_view = view[(view['ACTUAL_ARRIVAL_TIME'] >= start_time) & (view['ACTUAL_ARRIVAL_TIME'] <= end_time)]

In [230]:
tsp_view.sort_values('ACTUAL_ARRIVAL_TIME', ascending = True)

,CALENDAR_ID,SERVICE_ABBR,DATE,ROUTE_ABBR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,ROUTE_STOP_SEQUENCE,TRIP_EDGE,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ACTUAL_DEPARTURE_TIME,ADHERENCE,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT
613654,120250505,1,2025-05-05,50,429627,0,FROM DOWNTOWN,15,0,12:00:00,12:00:10,12:00:10,-0.166666,16.0,13.133333,-2.866667,0,0,1
552520,120250210,1,2025-02-10,50,417901,0,TO DOWNTOWN,15,0,12:05:00,12:00:17,12:04:54,0.100000,15.0,15.816666,0.816666,0,0,1
547546,120250203,1,2025-02-03,50,417968,0,FROM DOWNTOWN,14,0,11:55:00,12:00:26,12:00:26,-5.433333,15.0,18.433333,3.433333,0,0,1
547141,120250203,1,2025-02-03,50,417796,0,FROM DOWNTOWN,17,0,11:57:00,12:00:31,12:03:12,-6.200000,15.0,21.450000,6.450000,0,1,0
552669,120250210,1,2025-02-10,50,417968,0,FROM DOWNTOWN,15,0,12:00:00,12:00:32,12:00:32,-0.533333,16.0,14.916666,-1.083334,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613748,120250505,1,2025-05-05,50,429643,0,FROM DOWNTOWN,13,1,1900-01-01 00:15:00,23:59:06,1900-01-01 00:16:18,-1.300000,30.0,28.666666,-1.333334,0,0,1
613747,120250505,1,2025-05-05,50,429642,0,TO DOWNTOWN,13,2,1900-01-01 00:05:00,23:59:06,23:59:06,5.900000,NaN,NaN,NaN,0,0,1
613594,120250505,1,2025-05-05,50,429575,0,FROM DOWNTOWN,15,0,23:57:00,23:59:14,23:59:14,-2.233333,30.0,30.566666,0.566666,0,0,1
618861,120250512,1,2025-05-12,50,429642,0,TO DOWNTOWN,13,2,1900-01-01 00:05:00,23:59:22,23:59:22,5.633333,NaN,NaN,NaN,0,0,1


In [236]:
tsp_view['ACTUAL_DEPARTURE_TIME'] = tsp_view['ACTUAL_DEPARTURE_TIME'].str[-8:]

C:\Users\KLSHA\AppData\Local\Temp\ipykernel_6304\3167865336.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tsp_view['ACTUAL_DEPARTURE_TIME'] = tsp_view['ACTUAL_DEPARTURE_TIME'].str[-8:]


In [240]:
tsp_view.sort_values('ACTUAL_DEPARTURE_TIME', ascending = True)

,CALENDAR_ID,SERVICE_ABBR,DATE,ROUTE_ABBR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,ROUTE_STOP_SEQUENCE,TRIP_EDGE,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ACTUAL_DEPARTURE_TIME,ADHERENCE,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT
547488,120250203,1,2025-02-03,50,417916,0,FROM DOWNTOWN,16,0,1900-01-01 00:00:00,1900-01-01 00:00:38,00:00:38,-0.633333,30.0,30.233333,0.233333,0,0,1
618708,120250512,1,2025-05-12,50,429575,0,FROM DOWNTOWN,15,0,23:57:00,1900-01-01 00:00:45,00:00:45,-3.750000,30.0,31.100000,1.100000,0,0,1
552610,120250210,1,2025-02-10,50,417916,0,FROM DOWNTOWN,16,0,1900-01-01 00:00:00,1900-01-01 00:00:53,00:00:53,-0.883333,30.0,32.616666,2.616666,0,0,1
613595,120250505,1,2025-05-05,50,429575,0,FROM DOWNTOWN,16,0,1900-01-01 00:00:00,1900-01-01 00:00:54,00:00:54,-0.900000,30.0,29.833333,-0.166667,0,0,1
547640,120250203,1,2025-02-03,50,417983,0,TO DOWNTOWN,13,2,1900-01-01 00:05:00,1900-01-01 00:01:58,00:01:58,3.033333,NaN,NaN,NaN,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547487,120250203,1,2025-02-03,50,417916,0,FROM DOWNTOWN,15,0,23:57:00,23:59:03,23:59:03,-2.050000,30.0,32.833333,2.833333,0,0,1
613747,120250505,1,2025-05-05,50,429642,0,TO DOWNTOWN,13,2,1900-01-01 00:05:00,23:59:06,23:59:06,5.900000,NaN,NaN,NaN,0,0,1
613594,120250505,1,2025-05-05,50,429575,0,FROM DOWNTOWN,15,0,23:57:00,23:59:14,23:59:14,-2.233333,30.0,30.566666,0.566666,0,0,1
618861,120250512,1,2025-05-12,50,429642,0,TO DOWNTOWN,13,2,1900-01-01 00:05:00,23:59:22,23:59:22,5.633333,NaN,NaN,NaN,0,0,1


In [244]:
true_tsp_view = tsp_view.sort_values('ACTUAL_DEPARTURE_TIME', ascending = False)

In [246]:
true_tsp_view['ACTUAL_DEPARTURE_TIME'] = true_tsp_view['ACTUAL_DEPARTURE_TIME'].str[-8:]

In [248]:
true_tsp_view

,CALENDAR_ID,SERVICE_ABBR,DATE,ROUTE_ABBR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,ROUTE_STOP_SEQUENCE,TRIP_EDGE,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ACTUAL_DEPARTURE_TIME,ADHERENCE,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT
618861,120250512,1,2025-05-12,50,429642,0,TO DOWNTOWN,13,2,1900-01-01 00:05:00,23:59:22,23:59:22,5.633333,NaN,NaN,NaN,0,0,1
613594,120250505,1,2025-05-05,50,429575,0,FROM DOWNTOWN,15,0,23:57:00,23:59:14,23:59:14,-2.233333,30.0,30.566666,0.566666,0,0,1
613747,120250505,1,2025-05-05,50,429642,0,TO DOWNTOWN,13,2,1900-01-01 00:05:00,23:59:06,23:59:06,5.900000,NaN,NaN,NaN,0,0,1
547487,120250203,1,2025-02-03,50,417916,0,FROM DOWNTOWN,15,0,23:57:00,23:59:03,23:59:03,-2.050000,30.0,32.833333,2.833333,0,0,1
608483,120250428,1,2025-04-28,50,429575,0,FROM DOWNTOWN,16,0,1900-01-01 00:00:00,23:58:59,23:58:59,1.016666,30.0,22.366666,-7.633334,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613595,120250505,1,2025-05-05,50,429575,0,FROM DOWNTOWN,16,0,1900-01-01 00:00:00,1900-01-01 00:00:54,00:00:54,-0.900000,30.0,29.833333,-0.166667,0,0,1
552610,120250210,1,2025-02-10,50,417916,0,FROM DOWNTOWN,16,0,1900-01-01 00:00:00,1900-01-01 00:00:53,00:00:53,-0.883333,30.0,32.616666,2.616666,0,0,1
618708,120250512,1,2025-05-12,50,429575,0,FROM DOWNTOWN,15,0,23:57:00,1900-01-01 00:00:45,00:00:45,-3.750000,30.0,31.100000,1.100000,0,0,1
547488,120250203,1,2025-02-03,50,417916,0,FROM DOWNTOWN,16,0,1900-01-01 00:00:00,1900-01-01 00:00:38,00:00:38,-0.633333,30.0,30.233333,0.233333,0,0,1


In [256]:
apr = true_tsp_view[true_tsp_view['DATE'] == '2025-04-28']

In [258]:
apr

,CALENDAR_ID,SERVICE_ABBR,DATE,ROUTE_ABBR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,ROUTE_STOP_SEQUENCE,TRIP_EDGE,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ACTUAL_DEPARTURE_TIME,ADHERENCE,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT
608483,120250428,1,2025-04-28,50,429575,0,FROM DOWNTOWN,16,0,1900-01-01 00:00:00,23:58:59,23:58:59,1.016666,30.0,22.366666,-7.633334,1,0,0
608482,120250428,1,2025-04-28,50,429575,0,FROM DOWNTOWN,15,0,23:57:00,23:57:04,23:57:04,-0.066666,30.0,24.750000,-5.250000,0,0,1
608634,120250428,1,2025-04-28,50,429642,0,TO DOWNTOWN,14,0,23:54:00,23:54:50,23:54:50,-0.833333,30.0,29.350000,-0.650000,0,0,1
608481,120250428,1,2025-04-28,50,429575,0,FROM DOWNTOWN,14,0,23:53:00,23:53:56,23:53:56,-0.933333,30.0,25.916666,-4.083334,0,0,1
608633,120250428,1,2025-04-28,50,429642,0,TO DOWNTOWN,15,0,23:50:00,23:51:20,23:51:20,-1.333333,30.0,29.116666,-0.883334,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608331,120250428,1,2025-04-28,50,429522,0,TO DOWNTOWN,17,0,1900-01-01 00:09:00,1900-01-01 00:15:08,00:15:08,-6.133333,30.0,33.883333,3.883333,0,1,0
608485,120250428,1,2025-04-28,50,429575,0,FROM DOWNTOWN,18,2,1900-01-01 00:14:00,1900-01-01 00:11:28,00:11:28,2.533333,NaN,NaN,NaN,0,0,1
608330,120250428,1,2025-04-28,50,429522,0,TO DOWNTOWN,18,1,1900-01-01 00:02:00,23:46:42,00:11:13,-9.216666,30.0,34.933333,4.933333,0,1,0
608484,120250428,1,2025-04-28,50,429575,0,FROM DOWNTOWN,17,0,1900-01-01 00:07:00,1900-01-01 00:03:32,00:07:53,-0.883333,30.0,24.950000,-5.050000,0,0,1


In [261]:
apr['ADJUSTED_ONTIME_COUNT'].sum()

394

In [263]:
not_apr = true_tsp_view[true_tsp_view['DATE'] != '2025-04-28']

In [265]:
not_apr['ADJUSTED_ONTIME_COUNT'].sum()

1612

In [267]:
true_tsp_view.isna().any()

CALENDAR_ID              False
SERVICE_ABBR             False
DATE                     False
ROUTE_ABBR               False
TRIP_ID                  False
OVERLOAD_ID              False
ROUTE_DIRECTION_NAME     False
ROUTE_STOP_SEQUENCE      False
TRIP_EDGE                False
SCHEDULED_TIME           False
ACTUAL_ARRIVAL_TIME      False
ACTUAL_DEPARTURE_TIME     True
ADHERENCE                 True
SCHEDULED_HDWY            True
ACTUAL_HDWY               True
HDWY_DEV                  True
ADJUSTED_EARLY_COUNT     False
ADJUSTED_LATE_COUNT      False
ADJUSTED_ONTIME_COUNT    False
dtype: bool

In [271]:
last_trips = true_tsp_view[true_tsp_view['TRIP_EDGE'] == 2]

In [273]:
last_trips

,CALENDAR_ID,SERVICE_ABBR,DATE,ROUTE_ABBR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,ROUTE_STOP_SEQUENCE,TRIP_EDGE,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ACTUAL_DEPARTURE_TIME,ADHERENCE,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT
618861,120250512,1,2025-05-12,50,429642,0,TO DOWNTOWN,13,2,1900-01-01 00:05:00,23:59:22,23:59:22,5.633333,NaN,NaN,NaN,0,0,1
613747,120250505,1,2025-05-05,50,429642,0,TO DOWNTOWN,13,2,1900-01-01 00:05:00,23:59:06,23:59:06,5.900000,NaN,NaN,NaN,0,0,1
608329,120250428,1,2025-04-28,50,429521,0,FROM DOWNTOWN,18,2,23:44:00,23:46:42,23:46:42,-2.700000,NaN,NaN,NaN,0,0,1
618555,120250512,1,2025-05-12,50,429521,0,FROM DOWNTOWN,18,2,23:44:00,23:41:09,23:41:09,2.850000,NaN,NaN,NaN,0,0,1
613441,120250505,1,2025-05-05,50,429521,0,FROM DOWNTOWN,18,2,23:44:00,23:40:39,23:40:39,3.350000,NaN,NaN,NaN,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552612,120250210,1,2025-02-10,50,417916,0,FROM DOWNTOWN,18,2,1900-01-01 00:14:00,1900-01-01 00:09:04,00:09:04,4.933333,NaN,NaN,NaN,0,0,1
613597,120250505,1,2025-05-05,50,429575,0,FROM DOWNTOWN,18,2,1900-01-01 00:14:00,1900-01-01 00:08:56,00:08:56,5.066666,NaN,NaN,NaN,0,0,1
608635,120250428,1,2025-04-28,50,429642,0,TO DOWNTOWN,13,2,1900-01-01 00:05:00,1900-01-01 00:02:16,00:02:16,2.733333,NaN,NaN,NaN,0,0,1
552762,120250210,1,2025-02-10,50,417983,0,TO DOWNTOWN,13,2,1900-01-01 00:05:00,1900-01-01 00:02:00,00:02:00,3.000000,NaN,NaN,NaN,0,0,1


In [285]:
true_tsp_view[true_tsp_view['ACTUAL_DEPARTURE_TIME'].isnull()]

,CALENDAR_ID,SERVICE_ABBR,DATE,ROUTE_ABBR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,ROUTE_STOP_SEQUENCE,TRIP_EDGE,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ACTUAL_DEPARTURE_TIME,ADHERENCE,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT
552739,120250210,1,2025-02-10,50,417980,0,FROM DOWNTOWN,13,1,21:15:00,21:02:38,NaN,NaN,30.0,NaN,NaN,0,0,0


In [287]:
true_tsp_view[true_tsp_view['ADHERENCE'].isnull()]

,CALENDAR_ID,SERVICE_ABBR,DATE,ROUTE_ABBR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,ROUTE_STOP_SEQUENCE,TRIP_EDGE,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ACTUAL_DEPARTURE_TIME,ADHERENCE,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT
552739,120250210,1,2025-02-10,50,417980,0,FROM DOWNTOWN,13,1,21:15:00,21:02:38,NaN,NaN,30.0,NaN,NaN,0,0,0


In [299]:
true_tsp_view['ACTUAL_DEPARTURE_TIME'].isnull().sum()

1

In [297]:
true_tsp_view['ADHERENCE'].isna().sum()

1

In [317]:
apr['ADHERENCE'].mean()

-3.196564359183673

In [319]:
not_apr['ADHERENCE'].mean()

-1.3289019339184307

In [321]:
apr['HDWY_DEV'].mean()

-0.024958630000000013

In [323]:
not_apr['HDWY_DEV'].mean()

0.27222540113708155

In [325]:
apr['HDWY_DEV'].max()

19.4

In [327]:
not_apr['HDWY_DEV'].max()

31.116666

In [331]:
apr['HDWY_DEV'].min()

-10.716667

In [333]:
not_apr['HDWY_DEV'].min()

-27.15

In [355]:
tsp_window = [
    (wego['DATE']>="2025-02-03") & (wego['DATE']<"2025-02-10"),
    (wego['DATE']>="2025-02-10") & (wego['DATE']<"2025-04-28"),
    (wego['DATE']>="2025-04-28") & (wego['DATE']<"2025-05-05"),
    (wego['DATE']>="2025-05-05") & (wego['DATE']<"2025-05-12"),
    (wego['DATE']>="2025-05-12")
]
phases = [
    "TSP_ON_UNCOND",
    "TSP_ON_SCHED_COND",
    "TSP_OFF",
    "TSP_ON_UNCOND",
    "TSP_ON_HDWY_COND"
]
wego["TSP_STATUS"] = np.select(tsp_window, phases, default="N/A")






adhere = [
    (wego['ADJUSTED_EARLY_COUNT']==1),
    (wego['ADJUSTED_LATE_COUNT']==1),
    (wego['ADJUSTED_ONTIME_COUNT']==1),
 ]
adhere_types = [
    "EARLY",
    "LATE",
    "ONTIME"
]
wego["ADHERE_STATUS"] = np.select(adhere, adhere_types, default="N/A")

headway = [
    (wego['HDWY_DEV']<0),
    (wego['HDWY_DEV']>0)
 ]
hdwy_types = [
    "BUNCHED",
    "GAPPED"
]
wego["HDWY_STATUS"] = np.select(headway, hdwy_types, default="N/A")

wego['HDWY_PERCENT'] = wego['HDWY_DEV'] / wego['SCHEDULED_HDWY']

wego['HDWY_PERCENT'] = wego['HDWY_PERCENT'].astype('float64')

hdwyper = [
    (wego['HDWY_PERCENT']>=0.5) & (wego['HDWY_PERCENT']<=1.5),
    (wego['HDWY_PERCENT']<.5) | (wego['HDWY_PERCENT']>1.5)
 ]
yesno = [
    "YES",
    "NO"
]
wego["GOOD_HDWY"] = np.select(hdwyper, yesno, default="N/A")

In [356]:
wego

,CALENDAR_ID,SERVICE_ABBR,ADHERENCE_ID,DATE,ROUTE_ABBR,BLOCK_ABBR,OPERATOR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,...,STOP_CANCELLED,PREV_SCHED_STOP_CANCELLED,IS_RELIEF,BLOCK_STOP_ORDER,DWELL_IN_MINS,TSP_STATUS,ADHERE_STATUS,HDWY_STATUS,HDWY_PERCENT,GOOD_HDWY
0,120230101,3,93549161,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0,0.0,0,2,8.133333,N/A,ONTIME,N/A,NaN,N/A
1,120230101,3,93549162,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0,0.0,0,5,0.000000,N/A,ONTIME,N/A,NaN,N/A
2,120230101,3,93549163,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0,0.0,0,11,0.000000,N/A,ONTIME,N/A,NaN,N/A
3,120230101,3,93549164,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0,0.0,0,13,0.000000,N/A,ONTIME,N/A,NaN,N/A
4,120230101,3,93549165,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0,0.0,0,18,2.150000,N/A,ONTIME,N/A,NaN,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618993,120250512,1,119531049,2025-05-12,50,8401,3077,432121,0,FROM DOWNTOWN,...,0,NaN,0,23,4.500000,TSP_ON_HDWY_COND,EARLY,N/A,NaN,N/A
618994,120250512,1,119531638,2025-05-12,50,9302,3246,432353,0,TO DOWNTOWN,...,0,0.0,0,2,27.283333,TSP_ON_HDWY_COND,ONTIME,N/A,NaN,N/A
618995,120250512,1,119531639,2025-05-12,50,9302,3246,432353,0,TO DOWNTOWN,...,0,NaN,0,3,0.133333,TSP_ON_HDWY_COND,ONTIME,N/A,NaN,N/A
618996,120250512,1,119531770,2025-05-12,50,9950,2448,432387,0,TO DOWNTOWN,...,0,0.0,0,2,8.166666,TSP_ON_HDWY_COND,ONTIME,GAPPED,0.088333,NO


In [353]:
wego.sort_values('CALENDAR_ID')

,CALENDAR_ID,SERVICE_ABBR,ADHERENCE_ID,DATE,ROUTE_ABBR,BLOCK_ABBR,OPERATOR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,...,STOP_CANCELLED,PREV_SCHED_STOP_CANCELLED,IS_RELIEF,BLOCK_STOP_ORDER,DWELL_IN_MINS,TSP_STATUS,ADHERE_STATUS,HDWY_STATUS,HDWY_PERCENT,GOOD_HDWY
0,120230101,3,93549161,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0,0.0,0,2,8.133333,N/A,ONTIME,N/A,NaN,N/A
352,120230101,3,93549519,2023-01-01,50,5003,2631,332596,0,TO DOWNTOWN,...,0,0.0,0,114,0.000000,N/A,ONTIME,GAPPED,0.146667,NO
351,120230101,3,93549518,2023-01-01,50,5003,2631,332596,0,TO DOWNTOWN,...,0,0.0,0,109,2.583333,N/A,ONTIME,GAPPED,0.140000,NO
350,120230101,3,93549517,2023-01-01,50,5003,2631,332596,0,TO DOWNTOWN,...,0,0.0,0,107,0.000000,N/A,ONTIME,GAPPED,0.105000,NO
349,120230101,3,93549516,2023-01-01,50,5003,2631,332596,0,TO DOWNTOWN,...,0,0.0,0,101,0.000000,N/A,ONTIME,GAPPED,0.046667,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618463,120250512,1,119524237,2025-05-12,50,5002,2109,429506,0,TO DOWNTOWN,...,0,0.0,0,223,0.000000,TSP_ON_HDWY_COND,ONTIME,GAPPED,0.033333,NO
618464,120250512,1,119524238,2025-05-12,50,5002,2109,429506,0,TO DOWNTOWN,...,0,0.0,0,228,0.000000,TSP_ON_HDWY_COND,ONTIME,BUNCHED,-0.011111,NO
618465,120250512,1,119524239,2025-05-12,50,5002,2109,429506,0,TO DOWNTOWN,...,0,NaN,0,235,0.000000,TSP_ON_HDWY_COND,ONTIME,N/A,NaN,N/A
618455,120250512,1,119524229,2025-05-12,50,5002,2109,429505,0,FROM DOWNTOWN,...,0,0.0,0,191,0.000000,TSP_ON_HDWY_COND,LATE,GAPPED,0.265556,NO


In [361]:
wego.columns.tolist()

['CALENDAR_ID',
 'SERVICE_ABBR',
 'ADHERENCE_ID',
 'DATE',
 'ROUTE_ABBR',
 'BLOCK_ABBR',
 'OPERATOR',
 'TRIP_ID',
 'OVERLOAD_ID',
 'ROUTE_DIRECTION_NAME',
 'TIME_POINT_ABBR',
 'ROUTE_STOP_SEQUENCE',
 'TRIP_EDGE',
 'LATITUDE',
 'LONGITUDE',
 'SCHEDULED_TIME',
 'ACTUAL_ARRIVAL_TIME',
 'ACTUAL_DEPARTURE_TIME',
 'ADHERENCE',
 'SCHEDULED_HDWY',
 'ACTUAL_HDWY',
 'HDWY_DEV',
 'ADJUSTED_EARLY_COUNT',
 'ADJUSTED_LATE_COUNT',
 'ADJUSTED_ONTIME_COUNT',
 'STOP_CANCELLED',
 'PREV_SCHED_STOP_CANCELLED',
 'IS_RELIEF',
 'BLOCK_STOP_ORDER',
 'DWELL_IN_MINS',
 'TSP_STATUS',
 'ADHERE_STATUS',
 'HDWY_STATUS',
 'HDWY_PERCENT',
 'GOOD_HDWY']

In [377]:
TSP_dates = wego[wego['TSP_STATUS'] != 'N/A']

In [379]:
TSP_dates

,CALENDAR_ID,SERVICE_ABBR,ADHERENCE_ID,DATE,ROUTE_ABBR,BLOCK_ABBR,OPERATOR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,...,STOP_CANCELLED,PREV_SCHED_STOP_CANCELLED,IS_RELIEF,BLOCK_STOP_ORDER,DWELL_IN_MINS,TSP_STATUS,ADHERE_STATUS,HDWY_STATUS,HDWY_PERCENT,GOOD_HDWY
546973,120250203,1,117339007,2025-02-03,50,318,3443,417223,0,TO DOWNTOWN,...,0,0.0,0,2,35.766666,TSP_ON_UNCOND,ONTIME,GAPPED,inf,NO
546974,120250203,1,117339008,2025-02-03,50,318,3443,417223,0,TO DOWNTOWN,...,0,NaN,0,3,0.000000,TSP_ON_UNCOND,EARLY,N/A,NaN,N/A
546975,120250203,1,117339821,2025-02-03,50,5000,329,417725,0,TO DOWNTOWN,...,0,0.0,0,2,7.550000,TSP_ON_UNCOND,EARLY,N/A,NaN,N/A
546976,120250203,1,117339822,2025-02-03,50,5000,329,417725,0,TO DOWNTOWN,...,0,0.0,0,5,0.000000,TSP_ON_UNCOND,ONTIME,N/A,NaN,N/A
546977,120250203,1,117339823,2025-02-03,50,5000,329,417725,0,TO DOWNTOWN,...,0,0.0,0,11,0.000000,TSP_ON_UNCOND,ONTIME,N/A,NaN,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618993,120250512,1,119531049,2025-05-12,50,8401,3077,432121,0,FROM DOWNTOWN,...,0,NaN,0,23,4.500000,TSP_ON_HDWY_COND,EARLY,N/A,NaN,N/A
618994,120250512,1,119531638,2025-05-12,50,9302,3246,432353,0,TO DOWNTOWN,...,0,0.0,0,2,27.283333,TSP_ON_HDWY_COND,ONTIME,N/A,NaN,N/A
618995,120250512,1,119531639,2025-05-12,50,9302,3246,432353,0,TO DOWNTOWN,...,0,NaN,0,3,0.133333,TSP_ON_HDWY_COND,ONTIME,N/A,NaN,N/A
618996,120250512,1,119531770,2025-05-12,50,9950,2448,432387,0,TO DOWNTOWN,...,0,0.0,0,2,8.166666,TSP_ON_HDWY_COND,ONTIME,GAPPED,0.088333,NO


In [409]:
Off = TSP_dates[TSP_dates['TSP_STATUS'] == 'TSP_OFF']

In [411]:
Off = Off.drop(columns=['ADHERENCE_ID', 'LONGITUDE', 'LATITUDE', 'IS_RELIEF', 'OVERLOAD_ID', 'OPERATOR', 'BLOCK_ABBR', 'TIME_POINT_ABBR', 'ROUTE_STOP_SEQUENCE'])

In [413]:
Off

,CALENDAR_ID,SERVICE_ABBR,DATE,ROUTE_ABBR,TRIP_ID,ROUTE_DIRECTION_NAME,TRIP_EDGE,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ACTUAL_DEPARTURE_TIME,...,ADJUSTED_ONTIME_COUNT,STOP_CANCELLED,PREV_SCHED_STOP_CANCELLED,BLOCK_STOP_ORDER,DWELL_IN_MINS,TSP_STATUS,ADHERE_STATUS,HDWY_STATUS,HDWY_PERCENT,GOOD_HDWY
607968,120250428,1,2025-04-28,50,428880,TO DOWNTOWN,1,15:10:00,14:42:05,15:12:26,...,1,0,0.0,2,30.35,TSP_OFF,ONTIME,N/A,NaN,N/A
607969,120250428,1,2025-04-28,50,428880,TO DOWNTOWN,2,15:25:00,15:20:36,15:26:00,...,1,0,NaN,3,5.40,TSP_OFF,ONTIME,N/A,NaN,N/A
607970,120250428,1,2025-04-28,50,429384,TO DOWNTOWN,1,04:39:00,03:58:01,04:39:43,...,1,0,0.0,2,41.70,TSP_OFF,ONTIME,N/A,NaN,N/A
607971,120250428,1,2025-04-28,50,429384,TO DOWNTOWN,0,04:45:00,04:44:31,04:44:31,...,1,0,0.0,6,0.00,TSP_OFF,ONTIME,N/A,NaN,N/A
607972,120250428,1,2025-04-28,50,429384,TO DOWNTOWN,0,04:52:00,04:52:06,04:52:06,...,1,0,0.0,12,0.00,TSP_OFF,ONTIME,N/A,NaN,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613075,120250504,3,2025-05-04,50,429677,FROM DOWNTOWN,0,22:55:00,22:55:59,22:55:59,...,1,0,0.0,399,0.00,TSP_OFF,ONTIME,GAPPED,0.035556,NO
613076,120250504,3,2025-05-04,50,429677,FROM DOWNTOWN,0,22:58:00,23:00:25,23:00:25,...,1,0,0.0,404,0.00,TSP_OFF,ONTIME,GAPPED,0.047778,NO
613077,120250504,3,2025-05-04,50,429677,FROM DOWNTOWN,0,23:01:00,23:02:33,23:02:33,...,1,0,0.0,407,0.00,TSP_OFF,ONTIME,GAPPED,0.050000,NO
613078,120250504,3,2025-05-04,50,429677,FROM DOWNTOWN,0,23:08:00,23:10:03,23:10:03,...,1,0,0.0,413,0.00,TSP_OFF,ONTIME,GAPPED,0.048333,NO


In [427]:
Off.groupby('ADHERE_STATUS')['TRIP_ID'].count().sort_values(ascending=False)

ADHERE_STATUS
ONTIME    4065
LATE       842
EARLY      113
N/A         92
Name: TRIP_ID, dtype: int64

In [417]:
Uncond = TSP_dates[TSP_dates['TSP_STATUS'] == 'TSP_ON_UNCOND']

In [425]:
Uncond.groupby('ADHERE_STATUS')['TRIP_ID'].count().sort_values(ascending=False)

ADHERE_STATUS
ONTIME    8263
LATE      1129
EARLY      697
N/A        147
Name: TRIP_ID, dtype: int64

In [429]:
Cond = TSP_dates[TSP_dates['TSP_STATUS'] == 'TSP_ON_SCHED_COND']

In [431]:
Cond.groupby('ADHERE_STATUS')['TRIP_ID'].count().sort_values(ascending=False)

ADHERE_STATUS
ONTIME    44749
LATE       6335
N/A        2837
EARLY      1952
Name: TRIP_ID, dtype: int64

In [433]:
hdwy_cond = TSP_dates[TSP_dates['TSP_STATUS'] == 'TSP_ON_HDWY_COND']

In [435]:
hdwy_cond.groupby('ADHERE_STATUS')['TRIP_ID'].count().sort_values(ascending=False)

ADHERE_STATUS
ONTIME    716
LATE       49
EARLY      34
N/A         5
Name: TRIP_ID, dtype: int64

In [437]:
Off.groupby('HDWY_STATUS')['TRIP_ID'].count().sort_values(ascending=False)

HDWY_STATUS
GAPPED     2036
BUNCHED    2025
N/A        1051
Name: TRIP_ID, dtype: int64

In [439]:
Uncond.groupby('HDWY_STATUS')['TRIP_ID'].count().sort_values(ascending=False)

HDWY_STATUS
BUNCHED    4094
GAPPED     4073
N/A        2069
Name: TRIP_ID, dtype: int64

In [441]:
Cond.groupby('HDWY_STATUS')['TRIP_ID'].count().sort_values(ascending=False)

HDWY_STATUS
GAPPED     22394
BUNCHED    20518
N/A        12961
Name: TRIP_ID, dtype: int64

In [443]:
hdwy_cond.groupby('HDWY_STATUS')['TRIP_ID'].count().sort_values(ascending=False)

HDWY_STATUS
GAPPED     326
BUNCHED    322
N/A        156
Name: TRIP_ID, dtype: int64

In [451]:
cond_percent = Cond[Cond['HDWY_PERCENT'] > 0]

In [455]:
cond_percent.groupby('ADHERE_STATUS')['TRIP_ID'].count().sort_values(ascending=False)

ADHERE_STATUS
ONTIME    17619
LATE       4537
EARLY       238
Name: TRIP_ID, dtype: int64

In [457]:
cond_percent.groupby('HDWY_STATUS')['TRIP_ID'].count().sort_values(ascending=False)

HDWY_STATUS
GAPPED    22394
Name: TRIP_ID, dtype: int64